In [1]:
#import findspark
import findspark
findspark.init()

In [82]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, when, lit, month, year, to_date
from pyspark.sql.types import StringType, DateType



In [3]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 22:29:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/27 22:29:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/27 22:29:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/27 22:29:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/27 22:29:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
## creating sparksession and giving an app name
spark = SparkSession(sc).builder.appName('sparkdf').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



22/11/27 22:30:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
df_rating = spark.read.csv('yelp_review.csv', header = True)

In [6]:
df_rating = spark.read.format("csv").option("header", "true").option("multiline","true").option('inferSchema', 'true').load('yelp_review.csv')#df.show(3)
#print(df_rating.printSchema())

In [7]:
df_rating.show(3)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester's is locat...|     0|    0|   0|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
only showing top 3 rows



In [8]:
print(df_rating.printSchema())

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)

None


In [9]:
def dataframe_clean(df):
    list = [1.0,2.0,3.0,4.0,5.0]
    stopwords_remover = StopWordsRemover(inputCol = 'text', outputCol = 'text_wsw')
    df = df.withColumn('date_format', df['date'].cast(DateType()))
    df = df.withColumn('label', df['stars'].cast('double'))
    df = df.filter(df.label.isin(list))
    df = df.dropna(subset = ['text', 'label'])
    df = df.withColumn("rating_class",when((df.label == 5.0) | (df.label == 4.0), lit("satisfied"))\
            .otherwise(lit("not satisfied")))

    return df

def space_remove(x):
    remove = [word for word in x if word != '']
    return remove

import re
#from nltk.corpus import stopwords

stopword_list = ['under', 'hasn', 'an', 'how', 'aren', 'did', 'out', 'place', 'him', 'of', 'its', 'were', 'they', 'doing', "it's", 'if', "you'd", 'there', "shouldn't", 'should', 'not', 'from', 'is', 'because', 'or', 'you', 'get', 'too', 'her', 'before', 'once', 'most', 'shan', 'hers', 'these', 'had', 'again', "you'll", 'here', 'y', 'was', 'where', 'themselves', 'some', "couldn't", "hasn't", 'more', 'couldn', 'his', 'needn', 'their', 'having', 'and', 'myself', "you've", 'been', 'all', 'own', 've', 'ma', 'isn', 'further', "didn't", 'it', 'doesn', 'yourselves', 'other', 'he', "that'll", 'but', "you're", 'why', 'nor', 'only', 'no', 'while', 'me', 'very', "don't", 'ain', 'a', 'the', 'she', "weren't", "shan't", 'really', 'above', 'haven', "should've", 'off', "wouldn't", 'any', 'this', "mightn't", 'wasn', 'be', 'yours', 'into', 'in', 'so', 'as', 'd', 're', 'then', 'will', 'wouldn', 'weren', 'm', 'when', 'being', 'to', 'same', 'your', 'ourselves', 'now', 'herself', 'have', 'through', 'that', 'at', "wasn't", 'my', "she's", 'what', 'after', 'shouldn', 'about', 'few', 'has', "haven't", 'won', 'hadn', 'until', "mustn't", 'food', 'during', 'up', 'for', "hadn't", 'which', 'than', 'below', 'like', 'both', 'i', 'mightn', 'each', 'himself', "doesn't", 'does', 'theirs', 'don', 'am', "isn't", 'against', 'we', 't', 'yourself', 'with', 'down', 'mustn', 'who', 'do', 'just', 'ours', 'them', "needn't", 'are', 'on', 'such', 'those', 'itself', 'whom', 'our', 's', 'll', 'can', 'o', "aren't", 'one', 'between', 'didn', 'by', "won't", 'over']

not_stopword = ['place', 'food','like', 'one', 'really', 'get']


In [10]:
def clean_rdd(rdd):
    return rdd.map(lambda x: (re.sub('\W+',' ', x[0]), x[1]))\
            .map(lambda x: ( x[0].lower().split(' '), x[1]))\
            .map(lambda x: (space_remove(x[0]), x[1]))\
            .map(lambda x: ([word for word in x[0] if word not in stopword_list], x[1]))


In [32]:
df_review  = dataframe_clean(df_rating).limit(5000).select('business_id', 'text', 'date_format','label','rating_class','date')
#df_review.show(5)
#df_review.printSchema()

In [33]:
df_review.show(5)

+--------------------+--------------------+-----------+-----+------------+----------+
|         business_id|                text|date_format|label|rating_class|      date|
+--------------------+--------------------+-----------+-----+------------+----------+
|AEx2SYEUJmTxVVB18...|Super simple plac...| 2016-05-28|  5.0|   satisfied|2016-05-28|
|VR6GpWIda3SfvPC-l...|Small unassuming ...| 2016-05-28|  5.0|   satisfied|2016-05-28|
|CKC0-MOWMqoeWf6s-...|Lester's is locat...| 2016-05-28|  5.0|   satisfied|2016-05-28|
|ACFtxLv8pGrrxMm6E...|Love coming here....| 2016-05-28|  4.0|   satisfied|2016-05-28|
|s2I_Ni76bjJNK9yG6...|Had their chocola...| 2016-05-28|  4.0|   satisfied|2016-05-28|
+--------------------+--------------------+-----------+-----+------------+----------+
only showing top 5 rows



In [13]:
## the clean RDD
rdd_base =  df_review.select('text', 'label',"rating_class").rdd.map(lambda x: (x[0],x[1]))
rdd_base_clean = clean_rdd(rdd_base)
#print(f'\n{rdd_base_clean.collect()[:5]}')


In [14]:
### 1. Find the distribution of reviews based on ratings/group using RDD

rdd_Q1 = rdd_base_clean.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y)

#print(f'\nDistribution of reviews based on rating is: \n{rdd_Q1.collect()}')


In [17]:
print(f'\nDistribution of reviews based on rating is: \n{rdd_Q1.collect()}')


Distribution of reviews based on rating is: 
[(5.0, 1608), (4.0, 1608), (3.0, 938), (1.0, 398), (2.0, 448)]


In [19]:
### 2.Find the average length of the comments for each rating /group

rdd_Q2 = rdd_base_clean.map(lambda x: (x[1],len(x[0])))\
        .mapValues(lambda x: (x,1)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
        .map(lambda x: (x[0], x[1][0]/x[1][1]))
print(f"\nthe average of comment's length for each rating is: \n{rdd_Q2.collect()}")



the average of comment's length for each rating is: 
[(5.0, 53.132462686567166), (4.0, 59.12624378109453), (3.0, 66.30063965884861), (1.0, 64.88442211055276), (2.0, 67.05803571428571)]


In [21]:
### 3. Find the frequent words (top 10) used in posting a review for each rating/group
def word_couple(x):
    result = []
    for i in x:
        result.append((i,1))
    return result
rdd_Q3 = rdd_base_clean.map(lambda x: (x[1],word_couple(x[0]))).reduceByKey(lambda x,y: x+y).sortByKey()

for elt in rdd_Q3.collect():
    list = [1.0,2.0,3.0,4.0,5.0]
    freq = sc.parallelize(elt[1]).reduceByKey(lambda x,y: x + y)
    print(f'\nThe Top 10 frequent words for {elt[0]} rating are: \
           # {sorted(freq.collect(), key = lambda x:x[1], reverse = True)[:10]}')



The Top 10 frequent words for 1.0 rating are:            # [('would', 205), ('service', 164), ('time', 160), ('back', 157), ('go', 147), ('us', 137), ('even', 126), ('never', 119), ('minutes', 109), ('said', 108)]

The Top 10 frequent words for 2.0 rating are:            # [('good', 232), ('would', 195), ('time', 181), ('service', 151), ('back', 148), ('ordered', 144), ('go', 124), ('got', 117), ('also', 111), ('people', 101)]

The Top 10 frequent words for 3.0 rating are:            # [('good', 631), ('would', 348), ('time', 339), ('service', 297), ('go', 274), ('great', 270), ('also', 260), ('pretty', 251), ('little', 249), ('de', 245)]

The Top 10 frequent words for 4.0 rating are:            # [('good', 1025), ('great', 683), ('time', 546), ('also', 486), ('would', 439), ('service', 424), ('go', 419), ('nice', 403), ('back', 395), ('little', 376)]

The Top 10 frequent words for 5.0 rating are:            # [('great', 819), ('good', 552), ('time', 519), ('go', 452), ('also', 438), 

In [24]:
### 4. Find which business got most reviews after joining yelp_reviews with yelp_business

df_business = spark.read.csv('yelp_business.csv', header = True)
df_business = df_business.filter(~df_business.categories.isin(['0','1']))

# join dataset 'yelp_reviews' and 'yelp_business'
df_join1 = df_review.join(df_business, df_rating.business_id == df_business.business_id, 'inner')
df_join1 = df_join1.withColumn("categories",when(df_join1.categories == 'Restaurants;Mexican'\
        , lit('Mexican;Restaurants'))
        .otherwise(df_join1.categories))

rdd_cat_rating = df_join1.select('text', 'label', "categories")\
        .rdd.map(lambda x: (x[2],1)).reduceByKey(lambda x,y: x+y)

print('Top 10 business with the most reviews are : %s'%sorted(rdd_cat_rating.collect()\
        , key = lambda x:x[1], reverse = True)[:10])


Top 10 business with the most reviews are : [('Mexican;Restaurants', 94), ('Coffee & Tea;Food', 33), ('Food;Coffee & Tea', 27), ('Restaurants;Pizza', 18), ('Pizza;Restaurants', 17), ('Chinese;Restaurants', 16), ('Restaurants;Italian', 16), ('Italian;Restaurants', 14), ('Restaurants;Burgers', 14), ('Cinema;Arts & Entertainment', 14)]


In [26]:
### 5. Find which business got most 5-star rating and find the which business got most 4-star rating so on

rdd_cat = df_join1.select('text', 'label', 'categories').rdd.\
        map(lambda x: ((x[2],x[1]),1)).reduceByKey(lambda x,y: x+y)

list = [1.0,2.0,3.0,4.0,5.0]
for a in list:
    rdd_inter = rdd_cat.filter(lambda x: x[0][1] == a)
    print(f'\nThe categories with the most rating {a} is:\
            {sorted(rdd_inter.collect(), key = lambda x: (x[0][1], x[1]), reverse = True)[:10]}')

##################################


The categories with the most rating 1.0 is:            [(('Mexican;Restaurants', 1.0), 12), (('Active Life;Golf', 1.0), 4), (('Home Services;Apartments;Real Estate', 1.0), 4), (('Restaurants;Pizza', 1.0), 4), (('Restaurants;Italian', 1.0), 3), (('Restaurants;Pizza;Italian;Sandwiches', 1.0), 2), (('Arts & Entertainment;Cinema', 1.0), 2), (('Hotels & Travel;Hotels;Event Planning & Services', 1.0), 2), (('Restaurants;Barbeque', 1.0), 2), (('Shopping Centers;Shopping', 1.0), 2)]

The categories with the most rating 2.0 is:            [(('Mexican;Restaurants', 2.0), 10), (('Chinese;Restaurants', 2.0), 4), (('Food;Coffee & Tea', 2.0), 3), (('Coffee & Tea;Food', 2.0), 3), (('Vietnamese;Restaurants', 2.0), 3), (("Shopping;Fashion;Men's Clothing", 2.0), 2), (('Bakeries;Food', 2.0), 2), (('Burgers;Fast Food;Restaurants', 2.0), 2), (('Sports Bars;American (Traditional);Restaurants;Bars;Nightlife', 2.0), 2), (('Comfort Food;Restaurants;Steakhouses;American (New)', 2.0), 2)]

The categories with t

In [85]:
### 6.Find the average monthly/yearly review count in yelp system

# change date type in the review dataframe => see function 'dataframe_clean'
# create new variable month and year

rdd_ym = df_review.select(col('date'), col('business_id'), month(col("date")).alias("month"), year(col("date")).alias("year")).rdd.map(lambda x: (x[0],x[2], x[3]))
print(rdd_ym.collect()[:10])

#rdd_m = 
rdd_y = rdd_ym. map(lambda x: (x[2],1)).reduceByKey(lambda x,y : x+y)
print(rdd_y.collect()[:10])


[('2016-05-28', 5, 2016), ('2016-05-28', 5, 2016), ('2016-05-28', 5, 2016), ('2016-05-28', 5, 2016), ('2016-05-28', 5, 2016), ('2014-09-24', 9, 2014), ('2012-05-11', 5, 2012), ('2015-10-27', 10, 2015), ('2013-02-09', 2, 2013), ('2016-04-06', 4, 2016)]
[(2016, 801), (2014, 792), (2012, 331), (2015, 803), (2013, 664), (2011, 401), (2010, 222), (2017, 719), (2009, 111), (2008, 28)]
